In [2]:
import pandas as pd
import numpy as np
import json

# Our First Bots:

In [3]:
%%writefile rock.py
def rock(observation, configuration):
    return 0

Overwriting rock.py


In [4]:
%%writefile paper.py

def paper(observation, configuration):
    return 1

Writing paper.py


In [5]:
%%writefile scissors.py

def scissors(observation, configuration):
    return 2

Writing scissors.py
